# Import and Globals

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# Dataset Creation Class

In [2]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        print(self.char_indices)
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
        print(self.indices_char)

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print(questions[0:10])
print(expected[0:10])
print("Total questions:", len(questions))

{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
{0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}
Generating data...
['    5+0', '   4+06', '   5+86', '   36+1', '  687+1', '  71+81', '   6+98', '   9+78', '789+608', '  2+759']
['5   ', '64  ', '73  ', '64  ', '787 ', '35  ', '95  ', '96  ', '1793', '959 ']
Total questions: 50000


# Vectorization

In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)

Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [4]:
print(x_train[1])

[[ True False False False False False False False False False False False]
 [False False False False False False False False  True False False False]
 [False False False False  True False False False False False False False]
 [False  True False False False False False False False False False False]
 [False False False False  True False False False False False False False]
 [False False False False False False  True False False False False False]
 [False False False False False False False  True False False False False]]


In [5]:
print(len(chars))

12


# Model Creation

In [6]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [7]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 1
1407/1407 [==============================] - 7s 5ms/step - loss: 1.7692 - accuracy: 0.3533 - val_loss: 1.5769 - val_accuracy: 0.4053
Q 55+32   T 87   ☒ 44  
Q 816+17  T 833  ☒ 880 
Q 505+42  T 547  ☒ 556 
Q 165+600 T 765  ☒ 710 
Q 6+867   T 873  ☒ 876 
Q 212+7   T 219  ☒ 221 
Q 9+518   T 527  ☒ 164 
Q 532+4   T 536  ☒ 446 
Q 6+323   T 329  ☒ 346 
Q 297+7   T 304  ☒ 100 

Iteration 2
1407/1407 [==============================] - 6s 4ms/step - loss: 1.3470 - accuracy: 0.4950 - val_loss: 1.1499 - val_accuracy: 0.5745
Q 704+262 T 966  ☒ 1024
Q 0+212   T 212  ☒ 225 
Q 4+121   T 125  ☒ 128 
Q 427+6   T 433  ☒ 435 
Q 383+81  T 464  ☒ 455 
Q 734+211 T 945  ☒ 905 
Q 81+6    T 87   ☒ 80  
Q 116+43  T 159  ☒ 158 
Q 61+901  T 962  ☒ 954 
Q 312+31  T 343  ☒ 335 

Iteration 3
1407/1407 [==============================] - 6s 5ms/step - loss: 1.0159 - accuracy: 0.6258 - val_loss: 0.9197 - val_accuracy: 0.6635
Q 928+49  T 977  ☒ 974 
Q 58+298  T 356  ☒ 354 
Q 104+76  T 180  ☒ 174 
Q 590+17  

1407/1407 [==============================] - 7s 5ms/step - loss: 0.0116 - accuracy: 0.9974 - val_loss: 0.0448 - val_accuracy: 0.9868
Q 348+482 T 830  ☑ 830 
Q 2+610   T 612  ☑ 612 
Q 75+20   T 95   ☑ 95  
Q 387+444 T 831  ☑ 831 
Q 293+445 T 738  ☑ 738 
Q 620+545 T 1165 ☑ 1165
Q 835+57  T 892  ☑ 892 
Q 386+91  T 477  ☑ 477 
Q 964+11  T 975  ☑ 975 
Q 681+686 T 1367 ☑ 1367

Iteration 23
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0255 - accuracy: 0.9927 - val_loss: 0.0048 - val_accuracy: 0.9992
Q 254+803 T 1057 ☑ 1057
Q 615+69  T 684  ☑ 684 
Q 804+64  T 868  ☑ 868 
Q 696+167 T 863  ☑ 863 
Q 750+61  T 811  ☑ 811 
Q 45+986  T 1031 ☑ 1031
Q 528+481 T 1009 ☑ 1009
Q 825+832 T 1657 ☑ 1657
Q 529+859 T 1388 ☑ 1388
Q 44+134  T 178  ☑ 178 

Iteration 24
1407/1407 [==============================] - 7s 5ms/step - loss: 0.0233 - accuracy: 0.9931 - val_loss: 0.0191 - val_accuracy: 0.9943
Q 91+798  T 889  ☑ 889 
Q 172+642 T 814  ☑ 814 
Q 719+365 T 1084 ☑ 1084
Q 6+751   T 757  ☑ 75